In [ ]:
# Connect to Azure Machine Learning Workspace 
import sys
import os
import json
import archai.common.azureml_helper as aml_helper

sys.path.append(os.path.realpath('scripts'))

from cnn_search_space import CNNSearchSpace

# make sure we have a scripts dir for the code to run our jobs.
import os
scripts_dir = "./scripts"
os.makedirs(scripts_dir, exist_ok=True)

config_file = "../.azureml/config.json"
config = json.load(open(config_file, 'r'))

for required_key in ['subscription_id', 'resource_group', 'workspace_name', 'storage_account_key', 'storage_account_name']:
    if not required_key in config:
        print(f"### Error: please add a {required_key} to {config_file}")

storage_account_key = config['storage_account_key']    
storage_account_name = config['storage_account_name']

In [3]:
# Get a handle to the workspace
ml_client = aml_helper.get_aml_client_from_file(config_path=config_file)

Found the config file in: ..\.azureml\config.json


In [4]:
# Create cpu cluster for running the search
cpu_compute_name = "nas-cpu-cluster-D14-v2"
aml_helper.create_compute_cluster(ml_client, cpu_compute_name, size="Standard_D14_v2", location="westus")

# Create gpu cluster for running the search
gpu_compute_name = "nas-gpu-cluster-NC6"
aml_helper.create_compute_cluster(ml_client, gpu_compute_name, size="Standard_NC6", location="westus")

You already have a cluster named nas-cpu-cluster-D14-v2, we'll reuse it as is.
You already have a cluster named nas-gpu-cluster-NC6, we'll reuse it as is.


In [7]:
archai_job_env = aml_helper.create_environment_from_file(ml_client, conda_file="conda.yaml", version='0.1.1')

Environment with name aml-archai is registered to workspace, the environment version is 0.1.1


In [8]:
from azure.ai.ml.entities import AzureBlobDatastore
from azure.ai.ml.entities._credentials import AccountKeyConfiguration

# Create a blob store container for storing our NAS generated models in.
blob_store = AzureBlobDatastore(
    name=storage_account_name,
    description="Datastore pointing to our models blob container.",
    account_name=storage_account_name,
    container_name="models",
    credentials=AccountKeyConfiguration(
        account_key=storage_account_key
    ),
)

ml_client.create_or_update(blob_store)

AzureBlobDatastore({'type': <DatastoreType.AZURE_BLOB: 'AzureBlob'>, 'name': 'amlmnistmodels', 'description': 'Datastore pointing to our models blob container.', 'tags': {}, 'properties': {}, 'print_as_yaml': True, 'id': '/subscriptions/c8b7f913-60fb-4759-a310-fc5630e56f99/resourceGroups/snpe-aml-rg/providers/Microsoft.MachineLearningServices/workspaces/snpe-aml-workspace/datastores/amlmnistmodels', 'Resource__source_path': None, 'base_path': 'd:\\git\\microsoft\\archai\\docs\\advanced_guide\\cloud\\notebooks\\distributed_search', 'creation_context': None, 'serialize': <msrest.serialization.Serializer object at 0x0000014E5B267220>, 'credentials': {'type': 'account_key'}, 'container_name': 'models', 'account_name': 'amlmnistmodels', 'endpoint': 'core.windows.net', 'protocol': 'https'})

In [11]:
from status import ArchaiStatus

# make sure the status table exists.
status = ArchaiStatus(storage_connection_string)
status.get_all_status_entities()

AttributeError: 'ArchaiStatus' object has no attribute 'get_status_table_service'